## Opening a Sushi Restaurant in Beer Sheva, Israel

#### Introduction

Mrs. Name wishes to expand her Sushi franchise into Beer Sheva, Israel. The purpose of this work to help her find suitable locations in Beer Sheva for Mrs. Name's restaurant. Unfortunately, Mrs. Name has some strict specifications that we will have to consider upon searching for a suitable venue:
1. The propety must be suitable for a restaurant business.
2. The property must be 50-150 sqm in size.
3. The property must be located on the first floor.
4. The property's monthly rent fees must not exceed 20,000 NIS.
5. The property must be located as far from its competitors as possible.

#### Data Overview

The final product will be a list of suitable locations. To produce this list, we will use the Foursquare API and available data from the Beer Sheva Municipality, Google Maps, The Israel Bureau of Statistics, local real estate websites. 

#### Methodology

Stage 1:
Collecting the data and building the dataframes.

Stage 2:
Mapping the dataframes.

Stage 3:
Ranking properties based on their proximity to competitors.

#### Stage 1 - Collecting Data and Building Datasets

In [1076]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


An official open data website of the Beer Sheva municipality, offers different statistical data on the city has databases complete with neighborhoods, street names, house numbers and coordinates. We will combine them into a single dataset.

https://www.beer-sheva.muni.il/OpenData/Lists/Packages/CustomDispForm.aspx?ID=171


https://www.beer-sheva.muni.il/OpenData/Lists/Packages/CustomDispForm.aspx?ID=152


Let's download the datasets from the links above, and load them onto pandas dataframes.

In [1077]:
df_Addresses = pd.read_csv(r'C:\Users\evg28\Downloads\addresses.csv')
df_Addresses.head()

,StreetCode,HouseNuber,cityCode,streetName,letter,lat,lon
0,102,18.0,9000,דרך חברון,NaN,31.238322,34.795871
1,833,20.0,9000,המלאכה,NaN,31.237878,34.796022
2,102,155.0,9000,דרך חברון,NaN,31.253655,34.821186
3,102,159.0,9000,דרך חברון,NaN,31.254785,34.821342
4,531,5.0,9000,האורגים,NaN,31.254845,34.817054


Now, let's clean it up.

In [1078]:
#Deleting unnecessary columns
del df_Addresses['letter'], df_Addresses['cityCode'], df_Addresses['StreetCode']
#Editing column names
df_Addresses = df_Addresses.rename(columns={'HouseNuber':'House Number', 'streetName': 'Street Name', 'lat':'Latitude','lon':'Longitude'})

#Adding a column with the full address
df_Addresses.insert(2, 'Full Address', '')
#Combining street name and house number values into the new column
df_Addresses['Full Address'] = df_Addresses['Street Name'] + ' ' + df_Addresses['House Number'].astype(str)

df_Addresses.head()

,House Number,Street Name,Full Address,Latitude,Longitude
0,18.0,דרך חברון,דרך חברון 18.0,31.238322,34.795871
1,20.0,המלאכה,המלאכה 20.0,31.237878,34.796022
2,155.0,דרך חברון,דרך חברון 155.0,31.253655,34.821186
3,159.0,דרך חברון,דרך חברון 159.0,31.254785,34.821342
4,5.0,האורגים,האורגים 5.0,31.254845,34.817054


In [1079]:
df_Streetnames = pd.read_csv(r'C:\Users\evg28\Downloads\street-names.csv')
df_Streetnames.head()

,שם ראשי,תואר,שם מישני,קבוצה,קבוצה נוספת,סוג,קוד,שכונה
0,אוסישקין,NaN,מנחם (מנדל),ד. הסטוריה יהודית,NaN,רחוב,801.0,א
1,אוסקר,פרופ',לוי,ב. תושבי העיר,יא. תרבות ומדע,כיכר,NaN,א
2,אחד העם,NaN,NaN,ד. הסטוריה יהודית,יא. תרבות ומדע,רחוב,802.0,א
3,אלי כהן,NaN,NaN,ט. לוחמי ישראל,NaN,צומת,613.0,א
4,אסלן,NaN,בינימין,ב. תושבי העיר,NaN,כיכר,NaN,א


Let's clean the second dataset.

In [1080]:
#Deleting unnecessary columns
del df_Streetnames['תואר'], df_Streetnames['קבוצה'], df_Streetnames['קבוצה נוספת'], df_Streetnames['סוג'], df_Streetnames['קוד'], df_Streetnames['שם מישני']
#Editing column names
df_Streetnames = df_Streetnames.rename(columns={'שכונה':'Neighborhood', 'שם ראשי': 'Street Name'})
df_Streetnames.head()

,Street Name,Neighborhood
0,אוסישקין,א
1,אוסקר,א
2,אחד העם,א
3,אלי כהן,א
4,אסלן,א


A quick review of the data reveals a substantial discrepency in the names of streets in the datasets. This will require a standartization if the values are to be merged into a single dataset. 

Luckily, while Beer Sheva has over a thousand street names, our relevant datasets (available venues for rent and competitors) will consist of a mere dozen rows at most, so we may want to avoid correcting hundreds of names, and rather standardize only the relevant ones.

Now let's find available venues for rent and load them onto a dataframe. 

For this purpose, we can check popular Israeli real estate billboards, starting with Yad2 (the most popular).

The link below leads to the results of the query entered on Yad2 with the parameters required by Mrs. Name:
https://www.yad2.co.il/realestate/commercial?area=22&dealType=1&property=13,20&price=-1-20000&squaremeter=100-150

Let's try to scrape it to collect the necessary data.

In [1081]:
#Importing libraries
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [1082]:
#Scraping the webpage
url_rent = 'https://www.yad2.co.il/realestate/commercial?area=22&dealType=1&property=20,13&price=5000--1'
page_rent = requests.get(url_rent)
soup = BeautifulSoup(page_rent.text, 'html.parser')
print(soup.find_all('h1'))
print(soup.find_all('h2'))
print(soup.find_all('img'))

[<h1 class="title">?Are you for real</h1>]
[<h2 class="sub-title">אנו מניחים שגולשים כאן בני אנוש<br/>אבל רק כדי לוודא, אתם יודעים כבר מה לעשות...</h2>]
[<img alt="לוגו יד2" class="logo" src="https://assets.yad2.co.il/yad2site/y2assets/images/header/yad2Logo.png"/>, <img alt="לוגו יד2" class="logo-mobile" src="https://assets.yad2.co.il/yad2site/y2assets/images/header/Yad2_logo_white2.svg"/>, <img alt="" class="robot-captcha" src="https://assets.yad2.co.il/y2_captcha_error_page/robot_checkup.png"/>]


Apparently (and as provided in the EULA: https://www.yad2.co.il/eula), Yad2 prohibits scraping their webpages. However, our query's results page returned only a dozen of possible locations, so they may still be copied manually and used to create a dataframe. 

Moreover, after examining each location, we might find some of them unsuitable, while others might lack critical data, such as an address, as is often the case with ads that are posted by realters, which might shorten our dataset even more.

Unfortunately, Yad2 lacks "restaurants" in its property types, and has yielded only a few suitable venues. If so, let us expand our search to other sources to fill our dataframe.   

Another popular Israeli billboard is called "Homeless". Unlike Yad2, it does include "restaurants/coffee shops" as part of its property types. The link below is the results page of our query on Homeless, which has yielded one result:

https://www.homeless.co.il/iski/city=%d7%91%d7%90%d7%a8%20%d7%a9%d7%91%d7%a2$$inumber3=14,8$$inumber4=5$$flong3_1=1000000

A third popular billboard has yilded no results at all:

https://nadlan.ynet.co.il/search/commercial-for-rent/%D7%91%D7%90%D7%A8%20%D7%A9%D7%91%D7%A2,%20%D7%99%D7%A9%D7%A8%D7%90%D7%9C?pts=1181&maxp=20000

In [1083]:
venues_for_rent = pd.DataFrame({'Available Properties':['Venue1','Venue2','Venue3','Venue4'],
                               'Full Address':['סוקולוב 43.0','התקוה 10.0','בן צבי יצחק 10.0','אפרים לרון 112.0'],
                               'Cost':['NaN','NaN','NaN','NaN'],
                               'Size':['120','150','91','130']})


venues_for_rent

,Available Properties,Full Address,Cost,Size
0,Venue1,סוקולוב 43.0,NaN,120
1,Venue2,התקוה 10.0,NaN,150
2,Venue3,בן צבי יצחק 10.0,NaN,91
3,Venue4,אפרים לרון 112.0,NaN,130


Now, let's check out the locations of the competitors and load them onto a dataframe. 

A query on Trip Advisor returned two results:
https://www.tripadvisor.com/Restaurants-g297741-c38-Beersheba_Southern_District.html.

A query on Foursquare City Guide returned a handful of results.
https://foursquare.com/explore?mode=url&near=Beersheba%2C%20Israel&nearGeoId=72057594038223466&q=Sushi%20Restaurant

A similar query on Google Maps faired better with over a dozen hits, including the ones found on Foursquare:
https://www.google.co.il/maps/search/%D7%A1%D7%95%D7%A9%D7%99,+%D7%91%D7%90%D7%A8+%D7%A9%D7%91%D7%A2%E2%80%AD/@31.2613844,34.8604131,12z/data=!3m1!4b1

Like Yad2, Trip Advisor, and Google don't allow scraping, requiring us to build another dataframe manually. 

In [1084]:
competitors = pd.DataFrame({'Full Address': ['הרצל 29.0','שד דוד טוביהו 125.0','קקל 18.0',
                         'תל חי 18.0','שד חן 8.0', 'ברוך קטינקא 2.0', 
                         'דרך השלום 52.0','ההנדסה 1.0','יצחק אבינו 5.0',
                         'שד ירושלים 2.0','שד רגר 28.0','חיטה 1.0'],
       'Restaurant': ['Izakaya','Japanica','Sushi Bar','SFTH',
                      'Natali\'s Sushi','Kampai Street Wok', 
                      'Sushi night bar','Kampai',
                      'Asyatica','Mukpatzin','Hanoi','Duohi']})


competitors

,Full Address,Restaurant
0,הרצל 29.0,Izakaya
1,שד דוד טוביהו 125.0,Japanica
2,קקל 18.0,Sushi Bar
3,תל חי 18.0,SFTH
4,שד חן 8.0,Natali's Sushi
5,ברוך קטינקא 2.0,Kampai Street Wok
6,דרך השלום 52.0,Sushi night bar
7,ההנדסה 1.0,Kampai
8,יצחק אבינו 5.0,Asyatica
9,שד ירושלים 2.0,Mukpatzin


Now let's combine everything into a single dataframe.

In [1085]:
df_competitors = competitors.merge(df_Addresses, how = 'inner', sort = False)
df_competitors

,Full Address,Restaurant,House Number,Street Name,Latitude,Longitude
0,הרצל 29.0,Izakaya,29.0,הרצל,31.239183,34.787364
1,תל חי 18.0,SFTH,18.0,תל חי,31.251679,34.788839
2,יצחק אבינו 5.0,Asyatica,5.0,יצחק אבינו,31.261188,34.795048
3,יצחק אבינו 5.0,Asyatica,5.0,יצחק אבינו,31.261021,34.795076
4,שד ירושלים 2.0,Mukpatzin,2.0,שד ירושלים,31.245673,34.782267
5,חיטה 1.0,Duohi,1.0,חיטה,31.235014,34.800343


It seems that over half of the restaurants didn't make it, this is probably due to the inconsistency in street names.

The missing addresses are:
"שד דוד טוביהו 125.0"
"קקל 18.0"
"שד חן 8.0"
"ברוך קטינקא 2.0"
"דרך השלום 52.0"
"ההנדסה 1.0"
"שד ירושלים 2.0"
"שד רגר 28.0"

Let's seek them out and standardize them to fill out our new dataframe.

In [1086]:
for i in df_Addresses['Street Name']:
    if 'טוביהו' in i:
        print('df_Addresses', i)
        break
for i in df_Streetnames['Street Name'].astype(str):
    if 'טוביהו' in i:
        print('df_Streetnames', i)
for i in competitors['Full Address']:
    if 'טוביהו' in i:
        print('competitors', i)

df_Addresses שד טוביהו דוד
df_Streetnames טוביהו
competitors שד דוד טוביהו 125.0


The result above shows a total discrepancy between the datasets. Now we must standardize the street names.

In [1087]:
df_Addresses = df_Addresses.replace(["שד טוביהו דוד"], "שד דוד טוביהו")
df_Streetnames = df_Streetnames.replace(["טוביהו"],"שד דוד טוביהו")

Let's repeat the process with the rest of the missing locations.

In [1088]:
for i in df_Addresses['Street Name']:
    if  'קרן קיימת' in i:
        print('df_Addresses:', i)
        break
for i in df_Streetnames['Street Name'].astype(str):
    if 'קק"ל' in i:
        print('df_Streetnames:', i)
for i in competitors['Full Address']:
    if 'קקל' in i:
        print('competitors', i)

df_Addresses: קרן קיימת לישראל
df_Streetnames: הדסה-קק"ל
df_Streetnames: קק"ל - ארה"ב
competitors קקל 18.0


We can see a naming mismatch here too: "קקל", "קרן קיימת לישראל" and "קק"ל". Let's standardize it.

In [1089]:
df_Addresses = df_Addresses.replace(['קרן קיימת לישראל'],'קקל')
df_Streetnames = df_Streetnames.replace(['הדסה-קק"ל'], 'קקל')
df_Streetnames = df_Streetnames.replace(['קק"ל - ארה"ב'], 'קקל')

Let's move on to "שד חן".

In [1090]:
for i in df_Addresses['Street Name']:
    if  'ח"ן' in i:
        print('df_Addresses:', i)
        break
for i in df_Streetnames['Street Name'].astype(str):
    if 'חן' in i:
        print('df_Streetnames:', i)
for i in competitors['Full Address']:
    if 'חן' in i:
        print('competitors', i) 

df_Addresses: ח"ן
df_Streetnames: ריחן
df_Streetnames: חן
competitors שד חן 8.0


In [1091]:
df_Addresses = df_Addresses.replace(['ח"ן'], 'שד חן')
df_Streetnames = df_Streetnames.replace(['חן'], 'שד חן')

Next is "ברוך קטינקא 2.0".

In [1092]:
for i in df_Addresses['Street Name']:
    if  'קטי' in i:
        print('df_Addresses:', i)
        break
for i in df_Streetnames['Street Name'].astype(str):
    if 'קטי' in i:
        print('df_Streetnames:', i)
for i in competitors['Full Address']:
    if 'קטי' in i:
        print('competitors', i)

df_Addresses: קטינקא ברוך
df_Streetnames: קטינקא
competitors ברוך קטינקא 2.0


In [1093]:
df_Addresses = df_Addresses.replace(['קטינקא ברוך'], 'ברוך קטינקא')
df_Streetnames = df_Streetnames.replace(['קטינקא'], 'ברוך קטינקא')

Next is "דרך השלום 52.0"

In [1094]:
for i in df_Addresses['Street Name']:
    if  'השלום' in i:
        print('df_Addresses:', i)
        break
for i in df_Streetnames['Street Name'].astype(str):
    if 'השלום' in i:
        print('df_Streetnames:', i)
for i in competitors['Full Address']:
    if 'השלום' in i:
        print('competitors', i) 

df_Addresses: השלום
df_Streetnames: השלום
competitors דרך השלום 52.0


In [1095]:
df_Addresses = df_Addresses.replace(['השלום'], 'דרך השלום')
df_Streetnames = df_Streetnames.replace(['השלום'], 'דרך השלום')

Next is "ההנדסה 1.0"

In [1096]:
for i in df_Addresses['Street Name']:
    if 'הנדסה' in i:
        print('df_Addresses:', i)
        break
for i in df_Streetnames['Street Name'].astype(str):
    if 'הנדסה' in i:
        print('df_Streetnames:', i)
for i in competitors['Full Address']:
    if 'הנדסה' in i:
        print('competitors:', i)    

df_Addresses: חיל הנדסה
df_Streetnames: חיל הנדסה
competitors: ההנדסה 1.0


In [1097]:
df_Addresses = df_Addresses.replace(['חיל הנדסה'], 'ההנדסה')
df_Streetnames = df_Streetnames.replace(['חיל הנדסה'], 'ההנדסה')

Next is "שד ירושלים 2.0"

In [1098]:
for i in df_Addresses['Street Name'].astype(str):
    if 'שד ירושלים' in i:
        print('df_Addresses', i)
        break
for i in df_Streetnames['Street Name'].astype(str):
    if 'ירוש' in i:
        print('df_Streetnames', i)
for i in competitors['Full Address']:
    if 'ירוש' in i:
        print('competitors', i)

df_Addresses שד ירושלים
df_Streetnames הצדיק מירושלים
df_Streetnames ירושלים
competitors שד ירושלים 2.0


In [1099]:
df_Streetnames = df_Streetnames.replace(['ירושלים'], 'שד ירושלים')

The last address is "שד רגר 28.0"

In [1100]:
for i in df_Addresses['Street Name']:
    if 'רגר' in i:
        print('df_Addresses', i)
        break
for i in df_Streetnames['Street Name'].astype(str):
    if 'רגר' in i:
        print('df_Streetnames', i)
for i in competitors['Full Address']:
    if 'רגר' in i:
        print('competitors', i)

df_Addresses שד רגר יצחק
df_Streetnames רגר
df_Streetnames גרגר הנחלים
competitors שד רגר 28.0


In [1101]:
df_Addresses = df_Addresses.replace(['שד רגר יצחק'], 'שד רגר')
df_Streetnames = df_Streetnames.replace (['רגר'], 'שד רגר')

Now let's combine the street name and house number columns into a full address column, and try to join the datasets again.

In [1102]:
df_Addresses['Full Address'] = df_Addresses['Street Name'] + ' ' + df_Addresses['House Number'].astype(str)

In [1065]:
df_comb = competitors.merge(df_Addresses, how = 'inner', sort = False)
df_comb = df_comb.merge(df_Streetnames, how = 'inner', sort = False)
df_comb

,Full Address,Restaurant,House Number,Street Name,Latitude,Longitude,Neighborhood
0,הרצל 29.0,Izakaya,29.0,הרצל,31.239183,34.787364,עיר עתיקה
1,שד דוד טוביהו 125.0,Japanica,125.0,שד דוד טוביהו,31.250628,34.771949,א
2,קקל 18.0,Sushi Bar,18.0,קקל,31.237890,34.792229,ד
3,קקל 18.0,Sushi Bar,18.0,קקל,31.237890,34.792229,עיר עתיקה
4,תל חי 18.0,SFTH,18.0,תל חי,31.251679,34.788839,א
5,שד חן 8.0,Natali's Sushi,8.0,שד חן,31.266633,34.777503,רמות
6,ברוך קטינקא 2.0,Kampai Street Wok,2.0,ברוך קטינקא,31.225106,34.802902,קרית יהודית
7,דרך השלום 52.0,Sushi night bar,52.0,דרך השלום,31.251343,34.808246,ג
8,ההנדסה 1.0,Kampai,1.0,ההנדסה,31.244601,34.812736,א. תעשיה
9,יצחק אבינו 5.0,Asyatica,5.0,יצחק אבינו,31.261188,34.795048,ד


Much better!

Now, let's do the same with "venues_for_rent" to get the coordinates of the available properties for rent.

In [1103]:
df_venues = venues_for_rent.merge(df_Addresses, how = 'inner', sort = False)
df_venues

,Available Properties,Full Address,Cost,Size,House Number,Street Name,Latitude,Longitude
0,Venue2,התקוה 10.0,NaN,150,10.0,התקוה,31.248209,34.799296
1,Venue3,בן צבי יצחק 10.0,NaN,91,10.0,בן צבי יצחק,31.243253,34.799042


Looks like we are missing two addresses. Let's check them out.

In [1105]:
for i in df_Addresses['Street Name'].astype(str):
    if 'לרון' in i:
        print(i)

As we can see, 'לרון' is absent from df_Addresses

In [1106]:
for i in df_Addresses['Full Address'].astype(str):
    if 'סוקולוב 43.0' in i:
        print(i)

Apparently, the address 'סוקולוב 43.0' is absent from df_Addresses.

Now, let's fill in missing values.

In [1107]:
df_Addresses = df_Addresses.append({'Full Address': 'סוקולוב 43.0', 'Latitude':'31.245394', 'Longitude':'34.789741'}, ignore_index = True)
df_Addresses = df_Addresses.append({'Full Address': 'אפרים לרון 112.0', 'Latitude':'31.230874', 'Longitude':'34.767214'}, ignore_index = True)

In [1108]:
for i in df_Addresses['Full Address'].astype(str):
    if 'לרון' in i:
        print(i)
for i in df_Streetnames['Street Name'].astype(str):
    if 'לרון' in i:
        print(i)

אפרים לרון 112.0
לרון


In [1109]:
#Standardizing df_Streetnames
df_Streetnames = df_Streetnames.replace(['לרון'], 'אפרים לרון')

In [1110]:
#Updating df_venues
df_venues = venues_for_rent.merge(df_Addresses, how = 'inner', sort = False)
df_venues

,Available Properties,Full Address,Cost,Size,House Number,Street Name,Latitude,Longitude
0,Venue1,סוקולוב 43.0,NaN,120,NaN,NaN,31.245394,34.789741
1,Venue2,התקוה 10.0,NaN,150,10.0,התקוה,31.2482,34.7993
2,Venue3,בן צבי יצחק 10.0,NaN,91,10.0,בן צבי יצחק,31.2433,34.799
3,Venue4,אפרים לרון 112.0,NaN,130,NaN,NaN,31.230874,34.767214


In [1111]:
for i in df_Streetnames['Street Name'].astype(str):
    if 'צבי' in i:
        print(i)

צבי
צביה המלכה
הצבי
בן צבי
צביה ויצחק
אורי צבי
צבי-רן


In [1112]:
#Standardizing df_Streetnames
df_Streetnames = df_Streetnames.replace(['בן צבי'], 'בן צבי יצחק')

In [1113]:
#Updating df_venues
#df_venues = venues_for_rent.merge(df_Addresses, how = 'inner', sort = False)
df_venues = df_venues.merge(df_Streetnames, how = 'inner', sort = False)
df_venues

,Available Properties,Full Address,Cost,Size,House Number,Street Name,Latitude,Longitude,Neighborhood
0,Venue1,סוקולוב 43.0,NaN,120,NaN,NaN,31.245394,34.789741,NaN
1,Venue4,אפרים לרון 112.0,NaN,130,NaN,NaN,31.230874,34.767214,NaN
2,Venue2,התקוה 10.0,NaN,150,10.0,התקוה,31.2482,34.7993,מ. אזרחי
3,Venue3,בן צבי יצחק 10.0,NaN,91,10.0,בן צבי יצחק,31.2433,34.799,מ. אזרחי


Two neighborhood values are still missing, let's fill them manually.

In [1114]:
df_venues.at[0, 'Neighborhood'] = "א"
df_venues.at[1, 'Neighborhood'] = "נווה זאב"
df_venues

,Available Properties,Full Address,Cost,Size,House Number,Street Name,Latitude,Longitude,Neighborhood
0,Venue1,סוקולוב 43.0,NaN,120,NaN,NaN,31.245394,34.789741,א
1,Venue4,אפרים לרון 112.0,NaN,130,NaN,NaN,31.230874,34.767214,נווה זאב
2,Venue2,התקוה 10.0,NaN,150,10.0,התקוה,31.2482,34.7993,מ. אזרחי
3,Venue3,בן צבי יצחק 10.0,NaN,91,10.0,בן צבי יצחק,31.2433,34.799,מ. אזרחי


Now, after cleaning the datasets, lets plot them onto maps.

In [1115]:
import folium
from geopy.geocoders import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

In [1116]:
df_venues['Latitude'] = df_venues['Latitude'].astype(float)
df_venues['Longitude'] = df_venues['Longitude'].astype(float)

In [1117]:
address = 'BeerSheva, Israel'

geolocator = Nominatim(user_agent="bs_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [1118]:
map_venues = folium.Map(location=[latitude, longitude], zoom_start=12)
for Neighborhood, Latitude, Longitude in zip(df_venues['Neighborhood'], df_venues['Latitude'], df_venues['Longitude']):
    label = '{}, {}'.format(Latitude, Longitude)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues)
map_venues

On the map above, we can see a cluster of three locations at the city center, and one isolated location in the south-west. Next let's plot the competitors.

In [1119]:
df_comb['Latitude'] = df_comb['Latitude'].astype(float)
df_comb['Longitude'] = df_comb['Longitude'].astype(float)

In [1120]:
map_comp = folium.Map(location=[latitude, longitude], zoom_start=12)
for Latitude, Longitude in zip(df_comb['Latitude'], df_comb['Longitude']):
    label = '{}, {}'.format(Latitude, Longitude)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_comp)
map_comp

### Report

As we can see by the spread of venues and competitors, the afrementioned cluster of venues is located in proximity to multiple competitors.

In contrast, Venue4 is isolated in the south-west of the city, with no other competitors around, which qualifies it as the most suitable location for Mrs. Name's purposes.